<a href="https://colab.research.google.com/github/asingh-ml/NER_Product_Brand/blob/master/train_keras_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.15.0'

In [3]:
import keras
print(keras.__version__)

2.2.5


Using TensorFlow backend.


In [4]:
!pip install q keras==2.2.4

     |████████████████████████████████| 317kB 12.5MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [30]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5421334450639613802
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2833628273221969977
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8586033466305302324
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2657821808315024769
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [31]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls "/content/drive/My Drive/Ner_data"

amazon_product_unknown.h5  data_unknown_brand	 glove.6B.50d.txt
data			   data_unknown_product  model


In [0]:
!cp "/content/drive/My Drive/Ner_data/data/glove.6B.50d.trimmed.npz" "data/glove.6B.50d.trimmed.npz"
!cp "/content/drive/My Drive/Ner_data/data/test_random.csv" "data/test_random.csv"
!cp "/content/drive/My Drive/Ner_data/data/train.tagged.txt" "data/train.tagged.txt"
!cp "/content/drive/My Drive/Ner_data/data/test.tagged.txt" "data/test.tagged.txt"
!cp "/content/drive/My Drive/Ner_data/data/tags.txt" "data/tags.txt"
!cp "/content/drive/My Drive/Ner_data/data/words.txt" "data/words.txt"
# !cp "/content/drive/My Drive/Ner_data/data/test_unknown_product.csv" "data/test_unknown_product.csv"
# !cp "/content/drive/My Drive/Ner_data/train.tagged.pro.txt" "data/train.tagged.txt"
# !cp "/content/drive/My Drive/Ner_data/test.tagged.pro.txt" "data/test.tagged.txt"
# !cp "/content/drive/My Drive/Ner_data/glove.6B.50d.txt" "data/glove/glove.6B.50d.txt"
# !cp "/content/drive/My Drive/Ner_data/amazon_product_unknown.h5" "results/saves/amazon_product_unknown.h5"

In [0]:
import math
import numpy as np
import tensorflow as tf

from keras.layers import Input, Bidirectional, LSTM, Embedding, Dense, Dropout, Lambda, Concatenate, CuDNNLSTM
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from model.callbacks import F1score
from keras.backend.tensorflow_backend import set_session
from model.data_utils import minibatches, pad_sequences
from model.crf_layer import ChainCRF
from keras import optimizers
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from model.callbacks import LossHistory

In [0]:
class BaseKerasModel(object):
    """Generic class for general methods that are not specific to NER"""

    def __init__(self, config):
        """Defines self.config and self.logger
        Args:
            config: (Config instance) class with hyper parameters,
                vocab and embeddings
        """
        self.config = config
        self.logger = config.logger
        self.model = None
        self.conf = tf.ConfigProto()
        self.conf.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
        self.conf.log_device_placement = True  # to log device placement (on which device the operation ran)
        self.sess = tf.Session(config=self.conf)
        set_session(self.sess)
        self.sess = None
        self.saver = None

    def batch_iter(self, train, batch_size, return_lengths=False):
        """
        Creates a batch generator for the dataset
        :param train: Dataset
        :param batch_size: Batch Size
        :param return_lengths: If True, generator returns sequence lengths. Used masking data during the evaluation step
        :return: (number of batches in dataset, data generator)
        """
        nbatches = (len(train) + batch_size - 1) // batch_size

        def data_generator():
            while True:
                for i, (words, labels) in enumerate(minibatches(train, batch_size)):

                    # word_ids = zip(*words)
                    word_ids, sequence_lengths = pad_sequences(words, 0, max_length=self.config.max_length)

                    if labels:
                        labels, _ = pad_sequences(labels, 0, max_length=self.config.max_length)
                        labels = [to_categorical(label, num_classes=self.config.ntags) for label in labels] # Change labels to one-hot

                    # build dictionary
                    inputs = {
                        "word_ids": np.asarray(word_ids),
                    }

                    if return_lengths:
                        yield(inputs, np.asarray(labels), sequence_lengths)

                    else:
                        yield (inputs, np.asarray(labels))

        return (nbatches, data_generator())

    def train(self, train, dev, show_history=False):
        batch_size = self.config.batch_size

        nbatches_train, train_generator = self.batch_iter(train, batch_size)
        nbatches_dev, dev_generator = self.batch_iter(dev, batch_size)


        _, f1_generator = self.batch_iter(dev, batch_size, return_lengths=True)
        f1 = F1score(f1_generator, nbatches_dev, self.run_evaluate)

        callbacks = self.gen_callbacks([f1])

        history = self.model.fit_generator(generator=train_generator,
                                           steps_per_epoch=nbatches_train,
                                           validation_data=dev_generator,
                                           validation_steps=nbatches_dev,
                                           epochs=self.config.nepochs,
                                           callbacks=callbacks) #, nbatches_train

        if show_history:
            print(history.history['f1'])
            pass


    def predict_words(self, words_raw):
        words = [self.config.processing_word(w) for w in words_raw]
        # if type(words[0]) == tuple:
        #     words = zip(*words)
        # char_ids, word_ids = words

        word_ids = np.asarray(words)
        s = word_ids.shape
        word_ids = word_ids.reshape(-1, s[0])
        inputs = [word_ids]

        one_hot_preds = self.model.predict_on_batch(inputs)
        #print("One hot preds: ", one_hot_preds)
        one_hot_preds = [a.flatten() for a in one_hot_preds.squeeze()] #Squeeze to remove unnecessary 1st dimension for batch size
        #print("One hot preds: ", one_hot_preds)

        pred_ids = np.argmax(one_hot_preds, axis=1)
        #print("Pred ids: ", pred_ids)

        preds = [self.idx_to_tag[idx] for idx in pred_ids]

        return preds

    def run_evaluate(self, data_generator, steps_per_epoch):
        accs = []
        label_true = []
        label_pred = []
        for i in range(steps_per_epoch):
            #try:
            x_true, y_true, sequence_lengths = next(data_generator)
            y_pred = self.model.predict_on_batch(x_true)

            for lab, lab_pred, length in zip(y_true, y_pred,
                                             sequence_lengths):
                lab = lab[:length]
                lab_pred = lab_pred[:length]

                lab = np.argmax(lab, axis=1)
                lab_pred = np.argmax(lab_pred, axis=1)
                accs += [a==b for (a, b) in zip(lab, lab_pred)]


                label_true.extend(lab)
                label_pred.extend(lab_pred)


        label_true = np.asarray(label_true)
        #print("Truths: ", label_true)
        label_pred = np.asarray(label_pred)
        #print("Preds: ", label_pred)

        acc = np.mean(accs)
        print("acc: ", 100*acc)

        micro_score = f1_score(label_true, label_pred, average='micro')
        print(' - micro f1: {:04.2f}'.format(micro_score * 100))

        macro_score = f1_score(label_true, label_pred, average='macro')
        print(' - macro f1: {:04.2f}'.format(macro_score * 100))

        weighted_score = f1_score(label_true, label_pred, average='weighted')
        print(' - weighted f1: {:04.2f}'.format(weighted_score * 100))

        print(classification_report(label_true, label_pred))
        return (micro_score, macro_score, weighted_score)

    def get_loss(self):
        return self._loss

    def __getattr__(self, name):
        return getattr(self.model, name)

    def get_optimizer(self):
        return self._optimizer

In [0]:
class BLSTMCRF(BaseKerasModel):

    def __init__(self, config):
        super(BLSTMCRF, self).__init__(config)
        self._loss = None #losses.sparse_categorical_crossentropy
        self._optimizer = optimizers.Nadam(lr=self.config.lr)
        self.idx_to_tag = {idx: tag for tag, idx in
                           self.config.vocab_tags.items()}

    def build(self):
        inputs = [] #Create input for Model

        # build word embeddings
        input_words = Input(shape=(None,), dtype='int32', name='word_ids')
        inputs.append(input_words)
        if self.config.embeddings is None:
            word_embeddings = Embedding(input_dim=self.config.nwords,
                                        output_dim=self.config.dim_word,
                                        mask_zero=True,
                                        name="word_embeddings")(input_words)
        else:
            word_embeddings = Embedding(input_dim=self.config.nwords,
                                        output_dim=self.config.dim_word,
                                        mask_zero=True,
                                        weights=[self.config.embeddings],
                                        trainable=self.config.train_embeddings,
                                        name="word_embeddings")(input_words)

        word_embeddings = Dropout(self.config.dropout)(word_embeddings)
        encoded_text = Bidirectional(LSTM(units=self.config.hidden_size_lstm, return_sequences=True),
                                     name="bidirectional_1")(word_embeddings)
        encoded_text = Dropout(self.config.dropout)(encoded_text)
        encoded_text = Bidirectional(LSTM(units=self.config.hidden_size_lstm, return_sequences=True),
                                     name="bidirectional_2")(encoded_text)
        encoded_text = Dropout(self.config.dropout)(encoded_text)
        encoded_text = TimeDistributed(Dense(self.config.ntags))(encoded_text)
        encoded_text = Dropout(self.config.dropout)(encoded_text)
        #encoded_text = Dense(100, activation='tanh')(encoded_text)

        if self.config.use_crf:
#             crf = CRF(self.config.ntags, sparse_target=False)
            crf = ChainCRF()
            self._loss = crf.loss
#             self._loss = crf.loss_function
            pred = crf(encoded_text)

        else:
            self._loss = 'categorical_crossentropy'
            pred = Dense(self.config.ntags, activation='softmax')(encoded_text)

        self.model = Model(inputs, pred)


    def gen_callbacks(self, callbacks_list):
        lrate = LearningRateScheduler(self.step_decay)
        callbacks_list.append(lrate)

        #loss_history = LossHistory(self.step_decay)
        #callbacks_list.append(loss_history)
        return callbacks_list


    def step_decay(self, epoch):
        initial_lrate = self.config.lr
        decay = self.config.lr_decay
        epochs_drop = self.config.epoch_drop
        lrate = initial_lrate * math.pow(decay,
                                         math.floor((1+epoch)/epochs_drop))
        return lrate

    def plot_history(self, history):
        plt.plot(history.losses)
        plt.title('model losses')
        plt.ylabel('Loss')
        plt.xlabel('epoch')
        plt.show()

In [0]:
from model.data_utils import Dataset
from model.config import Config
from keras.models import load_model

config = Config()

In [50]:
# build model
model = BLSTMCRF(config)
model.build()
model.compile(optimizer=model.get_optimizer(), loss=model.get_loss())  # , metrics=['categorical_accuracy']
print(model.summary())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_ids (InputLayer)        (None, None)              0         
_________________________________________________________________
word_embeddings (Embedding)  (None, None, 50)          2656900   
_________________________________________________________________
dropout_9 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 300)         242400    
_________________________________________________________________
dropout_10 (Dropout)         (None, None, 300) 

In [0]:
# create datasets
dev = Dataset(config.filename_dev, config.processing_word,
              config.processing_tag, config.max_iter)
train = Dataset(config.filename_train, config.processing_word,
                config.processing_tag, config.max_iter)

In [52]:
# train model
model.train(train, dev)

Epoch 1/8
3676/3676 [==============================] - 535s 146ms/step - loss: 6.1944 - val_loss: 5.1933
acc:  91.39426232200223
 - micro f1: 91.39
 - macro f1: 85.06
 - weighted f1: 90.91
              precision    recall  f1-score   support

           0       0.91      0.61      0.73    431137
           1       0.94      0.83      0.88    508493
           2       0.91      0.98      0.94   2653252

    accuracy                           0.91   3592882
   macro avg       0.92      0.80      0.85   3592882
weighted avg       0.91      0.91      0.91   3592882

Epoch 2/8
3676/3676 [==============================] - 527s 143ms/step - loss: 3.9530 - val_loss: 4.6765
acc:  93.84683382309801
 - micro f1: 93.85
 - macro f1: 89.93
 - weighted f1: 93.66
              precision    recall  f1-score   support

           0       0.95      0.75      0.84    431137
           1       0.94      0.86      0.90    508493
           2       0.94      0.98      0.96   2653252

    accuracy           

In [0]:
import os

In [0]:
# Create directory for storing model object
if not os.path.exists('./results/saves'):
    os.makedirs('./results/saves')

# Save model
model.save_weights('./results/saves/amazon_unknown_brand_product.h5')

In [42]:
print("Model Trained and saved")

Model Trained and saved


In [0]:
config = Config()

model = BLSTMCRF(config) #Word_BLSTM(config)

model.build()
model.compile(optimizer=model.get_optimizer(), loss=model.get_loss()) #, metrics=['acc']

model.load_weights('./results/saves/amazon_product_unknown.h5')

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7




From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.



From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



...

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
NumExpr defaulting to 2 threads.


From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_ops.py:2509: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [54]:
model

In [55]:
test = Dataset(config.filename_test, config.processing_word,
                        config.processing_tag, config.max_iter)

batch_size = config.batch_size
nbatches_test, test_generator = model.batch_iter(test, batch_size, return_lengths=True)

model.run_evaluate(test_generator, nbatches_test)

acc:  84.87002861265077
 - micro f1: 84.87
 - macro f1: 75.47
 - weighted f1: 83.43
              precision    recall  f1-score   support

           0       0.83      0.45      0.58    128002
           1       0.92      0.67      0.77    132973
           2       0.84      0.98      0.91    562437

    accuracy                           0.85    823412
   macro avg       0.86      0.70      0.75    823412
weighted avg       0.85      0.85      0.83    823412



(0.8487002861265077, 0.7547080830591738, 0.8343335789052635)

In [0]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [18]:
! pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=25649bb0d18a5b15b1ac2758d23d2d091a36eaa1f07f5116065d5a9efac27e80
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
import pandas as pd
from ast import literal_eval
from seqeval.metrics import classification_report

In [25]:
testdata = pd.read_csv('./data/test_unknown_brand.csv')
testdata['tag'] = testdata['tag'].apply(lambda x: literal_eval(x))
actual = []
predicted = []
count = 0
for idx, row in testdata.iterrows():
    count+=1
    tagged = row['tag']
    words, labels = [word for word, ta in tagged], [ta for word, ta in tagged]
    if not words or len(tagged) == 1:
        continue
    pred = model.predict_words(words)
    # print(words)
    # print(labels)
    # print(pred)
    labels = rreplace(' '.join(items for items in labels).replace('PRODUCT', 'B-PRODUCT', 1), 'PRODUCT',
                      'I-PRODUCT',
                      ' '.join(items for items in labels).replace('PRODUCT', 'B-PRODUCT', 1).count(
                          'PRODUCT') - 1).split()
    labels = rreplace(' '.join(items for items in labels).replace('BRAND', 'B-BRAND', 1), 'BRAND',
                          'I-BRAND',
                          ' '.join(items for items in labels).replace('BRAND', 'B-BRAND', 1).count(
                              'BRAND') - 1).split()

    pred = rreplace(' '.join(items for items in pred).replace('PRODUCT', 'B-PRODUCT', 1), 'PRODUCT',
                      'I-PRODUCT',
                      ' '.join(items for items in pred).replace('PRODUCT', 'B-PRODUCT', 1).count(
                          'PRODUCT') - 1).split()
    pred = rreplace(' '.join(items for items in pred).replace('BRAND', 'B-BRAND', 1), 'BRAND',
                          'I-BRAND',
                          ' '.join(items for items in pred).replace('BRAND', 'B-BRAND', 1).count(
                              'BRAND') - 1).split()


    # print(words)
    # print(labels)
    # print(pred)
    actual.append(labels)
    predicted.append(pred)
    if count % 10000 == 0:
        print(count, " Records processed")

10000  Records processed
20000  Records processed
30000  Records processed
40000  Records processed
50000  Records processed
60000  Records processed


In [21]:
# Random Test Data results
print(classification_report(actual, predicted))

           precision    recall  f1-score   support

  PRODUCT       0.87      0.62      0.73     91891
    BRAND       0.86      0.85      0.85     93730

micro avg       0.87      0.74      0.80    185621
macro avg       0.87      0.74      0.79    185621



In [23]:
# Unknown Product Test Data results
print(classification_report(actual, predicted))

           precision    recall  f1-score   support

  PRODUCT       0.17      0.07      0.10     27005
    BRAND       0.06      0.78      0.11      2018

micro avg       0.09      0.12      0.10     29023
macro avg       0.16      0.12      0.10     29023



In [26]:
# Unknown Brand Test Data results
print(classification_report(actual, predicted))

           precision    recall  f1-score   support

  PRODUCT       0.56      0.32      0.40     93416
    BRAND       0.56      0.55      0.55     65597

micro avg       0.56      0.41      0.47    159013
macro avg       0.56      0.41      0.47    159013

